In [ ]:
import dautil as dl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ch6util
from scipy.signal import medfilt
from IPython.display import HTML

In [ ]:
def shuffle(temp):
    indices = np.random.choice(start, n/12)
    sample = dl.collect.flatten([temp.values[i: i + 12] for i in indices])
    sample = medfilt(sample)
    df = pd.DataFrame({'TEMP': sample}, index=temp.index[:len(sample)])
    df = df.resample('A', how=np.median)

    return df

In [ ]:
temp = dl.data.Weather.load()['TEMP'].resample('M', how=np.median).dropna()
n = len(temp)
start = np.arange(n - 11)
np.random.seed(2609787)

In [ ]:
%matplotlib inline
context = dl.nb.Context('moving_boot')
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
cp = dl.plotting.CyclePlotter(sp.ax)
medians = []
slopes = []

for i in range(240):
    df = shuffle(temp)
    slopes.append(dl.ts.ar1(df.values.flatten())['slope'])
    medians.append(ch6util.diff_median(df, 2))

    if i < 5:
        cp.plot(df.index, df.values)
        
sp.label(ylabel_params=dl.data.Weather.get_header('TEMP'))

sns.distplot(medians, ax=sp.next_ax())
sp.label()

sns.distplot(slopes, ax=sp.next_ax())
sp.label()

mins = []
tops = []
xrng = range(30, len(medians))

for i in xrng:
    min, max = dl.stats.outliers(medians[:i])
    mins.append(min)
    tops.append(max)

    
cp = dl.plotting.CyclePlotter(sp.next_ax())
cp.plot(xrng, mins, label='5 %')
cp.plot(xrng, tops, label='95 %')
sp.label()
HTML(sp.exit())